In [2]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model

In [3]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model

### Read chem_xref

In [ ]:
chem_xref = read_csv("/Users/lizrad/Dev/iVnat/chem_xref.tsv" , sep='\t' , low_memory=False)
chem_xref

### Bigg IDs to dict key

In [ ]:
df_bigg = chem_xref.loc[2:10216,'XREF':'MNX_ID']
pivoted_bigg = df_bigg.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
pivoted_bigg
#pivoted_bigg=pivoted_bigg.drop(['metanetx'])
chem_xref_MNX_bigg_dict = pivoted_bigg.to_dict('index')
chem_xref_MNX_bigg_dict2 = chem_xref_MNX_bigg_dict['bigg']
chem_xref_MNX_bigg_dict2

### Bigg IDs to annotation field

In [ ]:
for met in model.metabolites:
    data=''
    if 'metanetx.chemical' in met.annotation:
        data=met.annotation['metanetx.chemical'] 
        
        for xref in chem_xref_MNX_bigg_dict2.keys():
            if chem_xref_MNX_bigg_dict2[xref]==data:
                met.annotation['bigg.metabolite']=xref
                print(met.annotation)


### Check for multiplicatives among the Bigg IDs

In [ ]:
dupl_bigg = {}
for met in model.metabolites:
    if "bigg.metabolite" not in met.annotation.keys():
        continue
    if met.id in ["cpd00261_c0", "cpd06227_c0", "cpd03572_c0", "cpd02446_c0","cpd02572_c0", "cpd01466_c0" ]:
        pass
    else:    
        if met.annotation["bigg.metabolite"] not in dupl_bigg.keys():
            dupl_bigg.setdefault(met.annotation["bigg.metabolite"],[met.id])
        else:
            dupl_bigg[met.annotation["bigg.metabolite"]].append(met.id)

In [ ]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        print(ls)

In [ ]:
for ls in dupl_bigg.values():
    if len(ls) > 1:
        #print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            for pair in zip(x,y):
                print(len(model.metabolites.get_by_id(pair[0]+"_"+pair[1]).reactions))

### Change the MNXM IDs to Bigg IDs

In [4]:
data_dir = "/Users/lizrad/Documents/Vibrio_folder"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model_new=cobra.io.read_sbml_model(join(data_dir, "iVnat_newbigg.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model_new

files found: 
iVnat_newbigg.xml, iVnatReport.html, iVnat_newbigg.json, iVnat_gapfill_glu.xml, iVnat_20180817.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x01110fb0f0
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


just choosing the first bigg id from the list (not ideal!!!!)

there are 63 metabolites which have multiple bigg id 

In [10]:
with_bigg=0
for met in model_new.metabolites:
    print(met.id)
    split, compartment=met.id.rsplit("_", 1)
    if "bigg.metabolite" not in met.annotation.keys():
        continue
    if met.id in ["cpd00261_c0", "cpd06227_c0", "cpd03572_c0", "cpd02446_c0","cpd02572_c0", "cpd01466_c0" ]:
        pass
    else: 
        with_bigg+=1
        if type(met.annotation["bigg.metabolite"]) == list:
            print("whole annotation:" ,met.annotation['bigg.metabolite'])
            
            met.id=met.annotation['bigg.metabolite'][0]+ '_' + compartment
        else: 
            met.id=met.annotation['bigg.metabolite']+ '_' + compartment
            
            
    #if met.id.startswith('cpd'):
        
        print(met.id)
      

        
        model_new.repair()
       # print(met.id)

4abz_c0
4abz_c0
2ahhmd_c0
whole annotation: ['2ahhmd', '6hmhptpp']
2ahhmd_c0
ppi_c0
ppi_c0
h_c0
h_c0
dhpt_c0
dhpt_c0
atp_c0
atp_c0
gly_c0
gly_c0
glucys_c0
glucys_c0
adp_c0
adp_c0
pi_c0
pi_c0
gthrd_c0
gthrd_c0
lpam_c0
lpam_c0
2mhop_c0
2mhop_c0
thmpp_c0
thmpp_c0
2mpdhl_c0
2mpdhl_c0
imp_c0
imp_c0
prpp_c0
prpp_c0
hxan_c0
hxan_c0
nad_c0
nad_c0
12ppd__R_c0
12ppd__R_c0
nadh_c0
nadh_c0
lald__D_c0
lald__D_c0
h2o_c0
whole annotation: ['h2o', 'oh1']
h2o_c0
CPD-576_c0
CE1787_c0
whole annotation: ['CE1787', 'CE5868', 'asp__L']
CE1787_c0
Carboxylates_c0
amet_c0
amet_c0
s_c0
s_c0
dtbt_c0
dtbt_c0
met__L_c0
met__L_c0
btn_c0
btn_c0
dad_5_c0
dad_5_c0
h_e0
h_e0
adn_e0
adn_e0
adn_c0
adn_c0
5-L-GLUTAMYL-L-AMINO-ACID_c0
5oxpro_c0
5oxpro_c0
L-Amino-Acids_c0
Lactosyl-Ceramides_c0
gal_c0
gal_c0
gluside_cho_c0
gluside_cho_c0
accoa_c0
accoa_c0
ser__L_c0
ser__L_c0
coa_c0
coa_c0
acser_c0
acser_c0
cmp_c0
cmp_c0
cdp_c0
cdp_c0
fum_e0
fum_e0
fum_c0
fum_c0
uacgam_c0
uacgam_c0
uagmda_c0
uagmda_c0
udp_c0
udp_c0
uaagmda_c0

galct__D_c0
whole annotation: ['galct__D', 'galctr__D']
galct__D_c0
5d4dglcr_c0
whole annotation: ['5d4dglcr', '5dh4dglc']
5d4dglcr_c0
ppbng_c0
ppbng_c0
hmbil_c0
hmbil_c0
actp_c0
actp_c0
Hexanoyl-ACPs_c0
3ooctACP_c0
3ooctACP_c0
MNXM162700_c0
MNXM146494_c0
glyald_c0
glyald_c0
glyc__R_c0
glyc__R_c0
3pg_c0
3pg_c0
CPD-11468_c0
CPD-11467_c0
gam_e0
gam_e0
gam6p_c0
gam6p_c0
MNXM9860_c0
dpcoa_c0
dpcoa_c0
g6p_B_c0
g6p_B_c0
2hxmp_c0
whole annotation: ['2hxmp', '2hymeph']
2hxmp_c0
2dglcn_c0
2dglcn_c0
3ddgc_c0
3ddgc_c0
prbatp_c0
prbatp_c0
glu5sa_c0
glu5sa_c0
1pyr5c_c0
1pyr5c_c0
MNXM163143_c0
3hmop_c0
3hmop_c0
fru_e0
fru_e0
f1p_c0
f1p_c0
gmp_c0
gmp_c0
2dmmq8_c0
2dmmq8_c0
MNXM162337_c0
13dpg_c0
13dpg_c0
N1aspmd_c0
N1aspmd_c0
h2o_e0
whole annotation: ['h2o', 'oh1']
h2o_e0
mi1p__D_c0
mi1p__D_c0
inost_c0
inost_c0
glu__L_e0
glu__L_e0
n8aspmd_c0
n8aspmd_c0
ibcoa_c0
ibcoa_c0
MNXM11912_c0
MNXM5129_c0
MNXM6297_c0
dhor__S_c0
dhor__S_c0
orot_c0
orot_c0
2mecdp_c0
2mecdp_c0
MNXM9694_c0
uppg3_c0
uppg3_c0
MNXM512

Alkyl-enyl-acyl-gly-P-EtOH-amines_c0
cdpea_c0
cdpea_c0
1-ALKENYL-2-ACYLGLYCEROL_c0
35cgmp_c0
35cgmp_c0
pram_c0
pram_c0
MNXM10122_c0
MNXM12665_c0
2p4c2me_c0
2p4c2me_c0
ak2g_cho_c0
whole annotation: ['ak2g_cho', 'ak2g_hs']
ak2g_cho_c0
Alkyl-acyl-gly-P-EtOH-amines_c0
MNXM6298_c0
MNXM162320_c0
cdpdodecg_c0
cdpdodecg_c0
D-BETA-D-HEPTOSE-17-DIPHOSPHATE_c0
gmhep1p_c0
gmhep1p_c0
prlp_c0
prlp_c0
MNXM8377_c0
MNXM11359_c0
thym_c0
thym_c0
2dr1p_c0
2dr1p_c0
3hpalmACP_c0
3hpalmACP_c0
3opalmACP_c0
3opalmACP_c0
MNXM5477_c0
BUTANEDIOL_c0
ACETOIN_c0
HC01161_c0
whole annotation: ['HC01161', 'apoACP']
HC01161_c0
2ddglcn_c0
whole annotation: ['2ddglcn', '2dh3dgal']
2ddglcn_c0
CPD-343_c0
MNXM8218_c0
MNXM9655_c0
lipoate_c0
whole annotation: ['lipoate', 'lipt']
lipoate_c0
CE5656_c0
whole annotation: ['CE5656', 'lipoamp']
CE5656_c0
cyan_c0
cyan_c0
galt_e0
galt_e0
galt1p_c0
galt1p_c0
3hpcoa_c0
3hpcoa_c0
3mbzald_c0
3mbzald_c0
m_tol_c0
m_tol_c0
taur_e0
taur_e0
4izp_c0
4izp_c0
forglu_c0
forglu_c0
CPD-14426_c0
CPD-

CPD-12885_c0
Citronellates_c0
ps160_c0
ps160_c0
pe160_c0
pe160_c0
CPD-14293_c0
doco13ac_c0
doco13ac_c0
5mcsn_c0
5mcsn_c0
MNXM146500_c0
thex2eACP_c0
thex2eACP_c0
drib_e0
drib_e0
drib_c0
drib_c0
MNXM426_c0
3hmoa_c0
3hmoa_c0
3mb2coa_c0
3mb2coa_c0
MNXM91691_c0
MNXM6822_c0
lys__D_c0
lys__D_c0
ERYTHRITOL_c0
D-ERYTHRITOL-4P_c0
cpd00261_c0
xu1p__D_c0
xu1p__D_c0
5apru_c0
5apru_c0
3hbutACP_c0
3hbutACP_c0
GIBBERELLIN_c0
GIBBERELLIN-2-O-BETA-D-GLUCOSIDE_c0
CPD-13834_c0
CPD-13835_c0
MNXM5794_c0
MNXM8772_c0
RS-3-Sulfolactate_c0
MNXM147675_c0
lac__L_e0
lac__L_e0
pydx_c0
pydx_c0
MNXM9621_c0
MNXM162249_c0
MNXM9382_c0
MNXM9691_c0
5odhf2a_c0
5odhf2a_c0
3hocoa_c0
3hocoa_c0
CPD-161_c0
6pgl_c0
6pgl_c0
VIOMYCIN_c0
VIOMYCIN-P_c0
u3aga_c0
u3aga_c0
3ophb_c0
3ophb_c0
MNXM73074_c0
pencilca_c0
pencilca_c0
acac_c0
acac_c0
CPD-216_c0
CPD-303_c0
zymst_c0
zymst_c0
fecost_c0
fecost_c0
CPD-466_c0
MNXM7957_c0
MNXM613_c0
MNXM5869_c0
MNXM5870_c0
CPD-485_c0
tym_c0
tym_c0
conialdh_c0
conialdh_c0
fer_c0
fer_c0
4mcat_c0
4mcat_

fruur_c0
fruur_c0
MNXM7961_c0
sql_c0
sql_c0
D-3-HYDROXYACYL-COA_c0
CPD-472_c0
CPD-13777_c0
5-UREIDO-4-IMIDAZOLE_CARBOXYLATE_c0
4-AMINO-5-IMIDAZOLE_CARBOXYLATE_c0
MNXM9653_c0
CPD-586_c0
pta_c0
pta_c0
thm_e0
thm_e0
MNXM9685_c0
MNXM9683_c0
lgt__S_c0
lgt__S_c0
5aprbu_c0
5aprbu_c0
S-Alkyl-L-Cysteine-S-Oxides_c0
2amac_c0
2amac_c0
Alkyl-Sulfenates_c0
3oxoadp_c0
3oxoadp_c0
CPD-10823_c0
CPD-10824_c0
MNXM5797_c0
MNXM8777_c0
Linalools_c0
MNXM6771_c0
MNXM6766_c0
MNXM146499_c0
MNXM162298_c0
2-hydroxyacyl-glutathiones_c0
2-Hydroxy-carboxylates_c0
HOMO-CIT_c0
inost_e0
inost_e0
MNXM2902_c0
CPD-463_c0
MNXM8023_c0
gal_bD_c0
gal_bD_c0
cpd06227_c0
malthp_c0
malthp_c0
co2_e0
co2_e0
MNXM8006_c0
5forthf_c0
5forthf_c0
D-PROLINE_c0
MNXM146746_c0
3dsphgn_c0
3dsphgn_c0
2hb_c0
whole annotation: ['2hb', '2hbut']
2hb_c0
Aryl-Dialkyl-Phosphate_c0
Dialkyl-phosphates_c0
mnt_c0
mnt_c0
mn2_c0
mn2_c0
BIOMASS_c0
colipa_c0
colipa_c0
ca2_c0
ca2_c0
kdolipid4_c0
kdolipid4_c0
MNXM8210_c0
MNXM8176_c0
cl_e0
cl_e0
MNXM8220_c0
ala

In [6]:
count = 0
for met in model_new.metabolites:
    if "bigg.metabolite"  in met.annotation.keys():
        if type(met.annotation["bigg.metabolite"]) == list:
            count+=1

In [7]:
len(model_new.metabolites)
with_bigg
count

1842

1089

63

### Save draft with mapped Bigg IDs

In [11]:
write_sbml_model(model_new, "/Users/lizrad/Documents/Vibrio_folder/iVnat_bigg.xml")